In [ ]:
!pip install datasets==2.10.1
!pip install transformers==4.27.3
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

In [ ]:
from datasets import load_dataset
from huggingface_hub import login
from datasets import interleave_datasets
import evaluate

In [ ]:
login(token='')

# youtube dataset

In [4]:
# datasets_List = ['aanosov/mfdp_youtube_20230523']

In [ ]:
train_youtube = load_dataset('aanosov/mfdp_youtube_20230523', split='train', streaming=True, use_auth_token=True)
train_youtube = train_youtube.remove_columns(['full_path', 'duration', 'source', 'if_completed'])
train_youtube = train_youtube.shuffle(seed=321, buffer_size=200)

test_youtube = load_dataset('aanosov/mfdp_youtube_20230523', split='train', streaming=True, use_auth_token=True)
test_youtube = test_youtube.remove_columns(['full_path', 'duration', 'source', 'if_completed'])
test_youtube = test_youtube.take(400)

In [ ]:
print(list(train_youtube.take(1)))
print()
print(list(test_youtube.take(1)))

# Golos dataset

In [ ]:
train_golos = load_dataset('aanosov/mfdp_golos09', split='train', streaming=True, use_auth_token=True)
train_golos = train_golos.remove_columns(['Unnamed: 0', 'duration', 'source', 'index'])
train_golos = train_golos.shuffle(seed=234, buffer_size=200)

In [ ]:
print(list(train_golos.take(1)))

# Sova audiobooks dataset

In [ ]:
train_sova = load_dataset('aanosov/mfdp_sova_part1', split='train', streaming=True, use_auth_token=True)
train_sova  = train_sova.remove_columns(['Unnamed: 0', 'duration', 'source'])
train_sova  = train_sova .shuffle(seed=234, buffer_size=200)

In [ ]:
print(list(train_sova.take(1)))

# Combining datasets

In [8]:
train = interleave_datasets([train_youtube, train_golos, train_sova])

In [ ]:
for sample in list(train.take(10)):
    print(sample)
    print()

--

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="ru", task="transcribe")

In [13]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="ru", task="transcribe")

In [14]:
from datasets import Audio

train = train.cast_column("audio", Audio(sampling_rate=16000))
test_youtube = test_youtube.cast_column("audio", Audio(sampling_rate=16000))

In [15]:
def prepare_dataset(batch):
    audio = batch["audio"]

    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    batch["labels"] = tokenizer(batch["answer"]).input_ids
    return batch

In [16]:
train = train.map(prepare_dataset)
test_youtube = test_youtube.map(prepare_dataset)

In [17]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


class DataCollatorSpeechSeq2SeqWithPadding:
    
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:

        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]

        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [18]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [19]:
metric_wer = evaluate.load("wer")
metric_cer = evaluate.load("cer")

In [20]:
# def normalize_string(x):
#     x = re.sub("[^0-9A-Za-zА-Яёа-яЁ ]", " ", x).lower()
#     return " ".join(x.split())

In [21]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric_wer.compute(predictions=pred_str, references=label_str)
    cer = 100 * metric_cer.compute(predictions=pred_str, references=label_str)

    return {"wer": wer, "cer": cer}

In [22]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [23]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [24]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/kaggle/working/whisper-small",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # increase by 2x for every 2x decrease in batch size
    learning_rate=5e-6,
    warmup_steps=1000,
    max_steps=6000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=400,
    save_steps=2000,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [25]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train,
    eval_dataset=test_youtube,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

-----

In [26]:
trainer.evaluate()

{'eval_loss': 1.2862707376480103,
 'eval_wer': 53.709481712601836,
 'eval_cer': 23.35554585029797,
 'eval_runtime': 300.3605,
 'eval_samples_per_second': 1.332,
 'eval_steps_per_second': 0.166}

In [ ]:
trainer.train()